## **Etapa 1: Obtencion de datos**

En esta práctica obtendrás los datos para predecir si un falcon 9 aterizará con éxito o no. Los datos serán recogidos mediante la API de SpaceX y nos aseguraremos de que siguen un formato adecuado para las fases siguientes. El siguiente es un ejemplo de un aterrizaje exitoso:

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/landing\_1.gif)


Aquí puedes ver varios ejemplos de aterrizajes fallidos:




![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/crash.gif)


La gran mayoría de aterrizajes fallidos son intencionados para la realización de diversos controles. Estos aterrizajes se llevan a cabo la mayoria en el oceano, veremos estadísticas al respecto.

## Objetivos



En esta práctica realizaras una GET request a la API de SpaceX. Tambien realizarás cierto data wrangling básico y limpieza de datos.

***


## Importación de librerias.

Nos importamos las siguientes librerías necesarias para el desarrollo de la práctica.

In [ ]:
import requests
import pandas as pd
import numpy as np
import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
#Las opciones anteriores aseguran que pandas muestre el df completo en jupyternotebooks

Realicemos ahora una petición GET a la API de SpaceX. La URL es la siguiente.


In [ ]:
spacex_url="https://api.spacexdata.com/v4/launches/past"

* Usando la librería requests usa el método GET y pasale como argumento URL anterior, guardala en una variable `response`  y realiza el print de `response.content`

In [ ]:
#INSERTA AQUÍ TU CÓDIGO
response = requests.get(spacex_url)
print(response.content)

Si la petición ha sido realizada correctamente el resultado del print debería ser similar al siguiente:


```
b'[{"fairings":{"reused":false,"recovery_attempt":false,"recovered":false, ...
```
(Recomendamos cerrar el resultado del print después de revisarlo para que google collab no sufra.)

Nuestra variable `response` contiene una grandísima cantidad de información sobre los SpaceX pero vamos a intentar formatearla para poder ver de que se trata exactamente y que nos interesa.


### Parte 1: Obtén los datos mediante peticiones GET y crea un dataframe

Hemos observado que la variable `response` tiene forma de diccionario (como la mayoría de respuestas a peticiones GET). Una forma efectiva de formatear dichos datos es transformarlos en un JSON y posteriormente en un dataframe de python:


In [ ]:
data=response.json()
data=pd.json_normalize(data)

*Utilizando el dataframe anterior muestra las 5 primeras filas*

In [ ]:
#INSERTA AQUÍ TU CÓDIGO
data[:5]

,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,flight_number,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,fairings.reused,fairings.recovery_attempt,fairings.recovered,fairings.ships,links.patch.small,links.patch.large,links.reddit.campaign,links.reddit.launch,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'merlin engine failure'}]",Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,1,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cd9ffd86e000604b32a,False,False,False,[],https://images2.imgbox.com/94/f2/NN6Ph45r_o.png,https://images2.imgbox.com/5b/02/QcxHUb5V_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-falcon-1-rocket-lost-launch.html,https://en.wikipedia.org/wiki/DemoSat,NaN
1,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 301, 'altitude': 289, 'reason': 'harmonic oscillation leading to premature engine shutdown'}]","Successful first stage burn and transition to second stage, maximum altitude 289 km, Premature engine shutdown at T+7 min 30 s, Failed to reach orbit, Failed to recover first stage",[],[],[],[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,2,DemoSat,2007-03-21T01:10:00.000Z,1174439400,2007-03-21T13:10:00+12:00,hour,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cdaffd86e000604b32b,False,False,False,[],https://images2.imgbox.com/f9/4a/ZboXReNb_o.png,https://images2.imgbox.com/80/a2/bkWotCIS_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc,Lk4zQ2wP-Nc,https://www.space.com/3590-spacex-falcon-1-rocket-fails-reach-orbit.html,https://en.wikipedia.org/wiki/DemoSat,NaN
2,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 140, 'altitude': 35, 'reason': 'residual stage-1 thrust led to collision between stage 1 and stage 2'}]",Residual stage 1 thrust led to collision between stage 1 and stage 2,[],[],[],"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006eeb1e4]",5e9e4502f5090995de566f86,3,Trailblazer,2008-08-03T03:34:00.000Z,1217734440,2008-08-03T15:34:00+12:00,hour,False,"[{'core': '5e9e289ef3591814873b2625', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",True,False,None,5eb87cdbffd86e000604b32c,False,False,False,[],https://images2.imgbox.com/6c/cb/na1tzhHs_o.png,https://images2.imgbox.com/4a/80/k1oAkY0k_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=v0w9p3U8860,v0w9p3U8860,http://www.spacex.com/news/2013/02/11/falcon-1-flight-3-mission-summary,https://en.wikipedia.org/wiki/Trailblazer_(satellite),NaN
3,2008-09-20T00:00:00.000Z,1.221869e+09,False,0.0,5e9d0d95eda69955f709d1eb,True,[],"Ratsat was carried to orbit on the first successful orbital launch of any privately funded and developed, liquid-propelled carrier rocket, the SpaceX Falcon 1",[],[],[],[5eb0e4b7b6c3bb0006eeb1e5],5e9e4502f5090995de566f86,4,RatSat,2008-09-28T23:15:00.000Z,1222643700,2008-09-28T11:15:00+12:00,hour,False,"[{'core': '5e9e289ef3591855dc3b2626', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': 

Observemos que la mayoría de datos interesantes no se muestran de forma explícita sino que son IDs. Estos IDs nos permiten obtener mas información haciendo pediciones GET a diferentes endpoints de la API.
Para mas infrmación: https://docs.spacexdata.com/

Revisando la documentación de la API nos hemos dado cuenta de que no son necesarias todas las columnas, por lo que vamos a reducir nuestro dataframe a lo crucial:

In [ ]:
#Reescribe el dataframe para quedarnos solo con las columnas 'rocket', 'payloads','success', 'launchpad', 'cores', 'flight_number', 'date_utc'
#INSERTA AQUÍ TU CÓDIGO:
columnas = ['rocket', 'payloads', 'success', 'launchpad', 'cores', 'flight_number', 'date_utc']
data = data[columnas]

In [ ]:
# Algunos cohetes tienen mas de una carga 'payload' o 'core'.
# Estos datos nos darán problemas en un futuro, entonces nos limitearemos a aquellos que solamente tienen un core o una carga (payload):
data = data[data['cores'].map(len)==1]
data = data[data['payloads'].map(len)==1]

# Otro problema que tenemos es que ahora tenemos listas de longitud 1.
# Las listas de python no son especialmente compatibles con SQL por lo que lo solucionaremos sacando el valor de la lista:
data['cores'] = data['cores'].map(lambda x : x[0])
data['payloads'] = data['payloads'].map(lambda x : x[0])

# El formato de la fecha no es especialmete cómodo por lo que lo formatearemos:
data['date'] = pd.to_datetime(data['date_utc']).dt.date

Si revisamos `rocket, payload,launchpad y cores` son IDs, estos IDs nos permiten hacer peticiones a diferetes endpoints y obtener más información. La información para cada variable se encuentra en los siguientes links:

*   Para <code>rocket</code> https://github.com/r-spacex/SpaceX-API/blob/master/docs/rockets/v4/one.md . De aquí nos quedaremos la versión del cohete propulsor.

*   Para <code>payload</code> https://github.com/r-spacex/SpaceX-API/blob/master/docs/payloads/v4/one.md . De aquí guardaremos la carga, la masa de esta carga, la orbita a la que fue enviada y el cliente.

*   Para <code>launchpad</code> https://github.com/r-spacex/SpaceX-API/blob/master/docs/launchpads/v4/one.md . Guardamos la latitud, longitud y nombre de las diferentes plataformas de lanzamiento.

*   Para <code>cores</code> https://github.com/r-spacex/SpaceX-API/blob/master/docs/cores/v4/one.md . Gaurdamos diferentes variables del nucleo.

Los datos serán guardados en listas y estas listas las utilizaremos para crear una nuevo dataframe con todos los datos:



In [ ]:
#Para rocket
BoosterVersion = []

#Para paylaod
PayloadMass = []
Payload = []
Orbit = []
Customers= []

#Para launchpad
LaunchSite = []
Longitude = []
Latitude = []

#Para core
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
Mission_Outcome = []
ReusedCount = []
Serial = []


Para completar las listas anteriores nos definiremos una lista de funciones para facilitar la obtención de los datos.

Empecemos con los cohetes:

In [ ]:
# DEFINICIÓN FUNCIÓN 1
def getBoosterVersion(data): #Definimos la función
    for version in data['rocket']: #Iteramos por cada fila de la columna rocket
       if version: #Comprobamos que no este vacía
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(version)) #Realizamos la petición GET (Aplicamos str() para poder concatenar)
        response = response.json() #Transformamos la respuesta en un diccionario.
        BoosterVersion.append(response['name']) #Añadimos a BoosterVersion el valor con key 'name'

De <code>launchpad</code> nos gustaría quedarnos con la longitud, la latitud y el nombre de la plataforma


In [ ]:
# DEFINICIÓN FUNCIÓN 2
#INSERTA AQUI TU CÓDIGO
def getLaunchSite(data):
    for lanzadera in data["launchpad"]:
      if lanzadera:
        response = requests.get("https://api.spacexdata.com/v4/launchpads/"+ str(lanzadera))
        response = response.json()
        LaunchSite.append(response['name'])
        Latitude.append(response['latitude'])
        Longitude.append(response['longitude'])
#Definimos la función getLaunchSite que recibe con argumento data
#Iteramos por cada fila de la columna launchpad
#Comprobamos que no este vacía
#Realizamos la petición GET (Aplicamos str() para poder concatenar)
#Transformamos la respuesta en un diccionario.
#Añadimos a Longitude el valor con key 'longitude'
#Añadimos a Latitude el valor con key 'latitude'
#Añadimos a LaunchSite el valor con key 'name'

De <code>payload</code> nos gustaría obtener la carga, el peso de la carga, el cliente y la órbita a la que fué enviada.

In [ ]:
# DEFINICIÓN FUNCIÓN 3
#INSERTA AQUI TU CÓDIGO
def getPayloadData(data):
    for carga in data["payloads"]:
      if carga:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+ str(carga))
        response = response.json()
        Payload.append(response['name'])
        PayloadMass.append(response['mass_kg'])
        Customers.append(response['customers'])
        Orbit.append(response['orbit'])

#Definimos la función getPayloadData
#Iteramos por cada fila de la columna payloads
#Comprobamos que no este vacía
#Realizamos la petición GET (Aplicamos str() para poder concatenar)
#Transformamos la respuesta en un diccionario.
#Añadimos a Payload el valor con key 'name'
#Añadimos a Customers el valor con key 'customers'
#Añadimos a PayloadMass el valor con key 'mass_kg'
#Añadimos a Orbit el valor con key 'orbit'

La función para completar los datos de `core` tiene una complicación extra por lo que la damos hecha:

In [ ]:
# DEFINICIÓN FUNCIÓN 4
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])


Las funciones anteriores rellenan las listas vacías, veamoslo en práctica. *Revisa que la lista BoosterVersion esta vacía*


In [ ]:
#INSERTA AQUI TU CÓDIGO
print(BoosterVersion)

[]


*Llama ahora la función `getBoosterVersion`.* (No devuelve nada así que no hace falta asignarle una variable. Solo instanciamos la funcion.)

In [ ]:
# LLAMADA A FUNCIÓN 1
#INSERTA AQUI TU CÓDIGO
getBoosterVersion(data)

*Muestra los primeros 5 valores de la lista `BoosterVersion`*

In [ ]:
#INSERTA AQUI TU CÓDIGO
print(BoosterVersion[0:5])

['Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 9']


*Aplica el resto de funciones:*
(No devuelven nada así que no hace falta asignarles una variable. Solo instanciamos a las funciones.)

In [ ]:
# LLAMADA A FUNCIÓN 2
#INSERTA AQUI TU CÓDIGO
getLaunchSite(data)

In [ ]:
# LLAMADA A FUNCIÓN 3
#INSERTA AQUI TU CÓDIGO
getPayloadData(data)

In [ ]:
# LLAMADA A FUNCIÓN 4
#INSERTA AQUI TU CÓDIGO
getCoreData(data)

Finalmente creamos un dataframe desde las listas anteriores. Primero convirtámolos en un diccionario:

In [ ]:
launch_dict = {'FlightNumber': list(data['flight_number']),
'Date': list(data['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Payload':Payload,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'LandingOutcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'Customers': Customers,
'Mission_Outcome':list(data['success']),
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}

Y ahora crea un dataframe de Pandas desde el diccionario. Llámalo `launch_data`

In [ ]:
#INSERTA AQUI TU CÓDIGO
launch_data = pd.DataFrame(launch_dict)
launch_data

,FlightNumber,Date,BoosterVersion,PayloadMass,Payload,Orbit,LaunchSite,LandingOutcome,Flights,GridFins,Reused,Legs,LandingPad,Block,Customers,Mission_Outcome,ReusedCount,Serial,Longitude,Latitude
0,1,2006-03-24,Falcon 1,20.0,FalconSAT-2,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,[DARPA],False,0,Merlin1A,167.743129,9.047721
1,2,2007-03-21,Falcon 1,NaN,DemoSAT,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,[DARPA],False,0,Merlin2A,167.743129,9.047721
2,4,2008-09-28,Falcon 1,165.0,RatSat,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,[SpaceX],True,0,Merlin2C,167.743129,9.047721
3,5,2009-07-13,Falcon 1,200.0,RazakSAT,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,[ATSB],True,0,Merlin3C,167.743129,9.047721
4,6,2010-06-04,Falcon 9,NaN,Dragon Qualification Unit,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[SpaceX],True,0,B0003,-80.577366,28.561857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,181,2022-08-28,Falcon 9,13260.0,Starlink 4-23 (v1.5),VLEO,KSC LC 39A,True ASDS,2,True,True,True,5e9e3033383ecb075134e7cd,5.0,[SpaceX],True,1,B1069,-80.603956,28.608058
168,182,2022-08-31,Falcon 9,13260.0,Starlink 3-4 (v1.5),VLEO,VAFB SLC 4E,True ASDS,7,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,[SpaceX],True,6,B1063,-120.610829,34.632093
169,185,2022-09-17,Falcon 9,13260.0,Starlink 4-34 (v1.5),VLEO,CCSFS SLC 40,True ASDS,6,True,True,True,5e9e3033383ecbb9e534e7cc,5.0,[SpaceX],True,5,B1067,-80.577366,28.561857
170,186,2022-09-24,Falcon 9,13260.0,Starlink 4-35 (v1.5),VLEO,CCSFS SLC 40,True ASDS,4,True,True,True,5e9e3033383ecbb9e534e7cc,5.0,[SpaceX],True,0,B1072,-80.577366,28.561857


Finalmente, muestra las primeras 5 filas del dataframe.

In [ ]:
#INSERTA AQUI TU CÓDIGO
launch_data[:5]

,FlightNumber,Date,BoosterVersion,PayloadMass,Payload,Orbit,LaunchSite,LandingOutcome,Flights,GridFins,Reused,Legs,LandingPad,Block,Customers,Mission_Outcome,ReusedCount,Serial,Longitude,Latitude
0,1,2006-03-24,Falcon 1,20.0,FalconSAT-2,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,[DARPA],False,0,Merlin1A,167.743129,9.047721
1,2,2007-03-21,Falcon 1,NaN,DemoSAT,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,[DARPA],False,0,Merlin2A,167.743129,9.047721
2,4,2008-09-28,Falcon 1,165.0,RatSat,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,[SpaceX],True,0,Merlin2C,167.743129,9.047721
3,5,2009-07-13,Falcon 1,200.0,RazakSAT,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,[ATSB],True,0,Merlin3C,167.743129,9.047721
4,6,2010-06-04,Falcon 9,NaN,Dragon Qualification Unit,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[SpaceX],True,0,B0003,-80.577366,28.561857


### Parte 2: Filtra el dataframe para que solo incluya los lanzamientos de falcon 9



El falcon 9 es una versión considerablemente más nueva y efectiva que el faclon 1. Dado que nos interesan los datos recientes y a futuro vamos a *eliminar los lanzamientos de `Falcon 1` del fataframe launch_data y llama al nuevo dataframe `data_falcon9`. Muestra las primeras 5 filas.*

In [208]:
# INSERTA AQUÍ TU CÓDIGO
condicion = launch_data["BoosterVersion"] == "Falcon 9"
data_falcon9 = launch_data[condicion]
data_falcon9[:5]

,FlightNumber,Date,BoosterVersion,PayloadMass,Payload,Orbit,LaunchSite,LandingOutcome,Flights,GridFins,Reused,Legs,LandingPad,Block,Customers,Mission_Outcome,ReusedCount,Serial,Longitude,Latitude
4,1,2010-06-04,Falcon 9,NaN,Dragon Qualification Unit,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[SpaceX],True,0,B0003,-80.577366,28.561857
5,2,2012-05-22,Falcon 9,525.0,COTS Demo Flight 2,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[NASA(COTS)],True,0,B0005,-80.577366,28.561857
6,3,2013-03-01,Falcon 9,677.0,CRS-2,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[NASA (CRS)],True,0,B0007,-80.577366,28.561857
7,4,2013-09-29,Falcon 9,500.0,CASSIOPE,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1.0,[MDA],True,0,B1003,-120.610829,34.632093
8,5,2013-12-03,Falcon 9,3170.0,SES-8,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[SES],True,0,B1004,-80.577366,28.561857


Ahora que hemos eliminado los lanzamientos de Falcon 1 la columna FlightNumber está desajustada. Vamos a volver a ordenarla:

In [197]:
data_falcon9.FlightNumber = list(range(1, data_falcon9.shape[0]+1))
data_falcon9.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Payload,Orbit,LaunchSite,LandingOutcome,Flights,GridFins,Reused,Legs,LandingPad,Block,Customers,Mission_Outcome,ReusedCount,Serial,Longitude,Latitude
4,1,2010-06-04,Falcon 9,NaN,Dragon Qualification Unit,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[SpaceX],True,0,B0003,-80.577366,28.561857
5,2,2012-05-22,Falcon 9,525.0,COTS Demo Flight 2,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[NASA(COTS)],True,0,B0005,-80.577366,28.561857
6,3,2013-03-01,Falcon 9,677.0,CRS-2,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[NASA (CRS)],True,0,B0007,-80.577366,28.561857
7,4,2013-09-29,Falcon 9,500.0,CASSIOPE,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1.0,[MDA],True,0,B1003,-120.610829,34.632093
8,5,2013-12-03,Falcon 9,3170.0,SES-8,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,[SES],True,0,B1004,-80.577366,28.561857


### Parte 3: Ajustes finales


Debajo podemos observar que nos faltan ciertos valores.

In [219]:
data_falcon9.isnull().sum()

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        0
Payload            0
Orbit              1
LaunchSite         0
LandingOutcome     0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad         0
Block              0
Customers          0
Mission_Outcome    1
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

* Lo óptimo sería no tener celdas vacías por lo que vamos a ver que podemos hacer para solucionar esta situación.
La columna <code>LandingPad</code> tiene valores `None`en los casos donde el cohete no tuvo intención de aterrizar por lo que no tenía ningún LandingPad asignado.

* Por otro lado, los valores innexistentes de `PayloadMass` si se pueden completar. *En este caso utilizaras la función `.mean()` para calcular la media y `.replace()` para reemplzar los valores `np.nan`.*


In [220]:
# INSERTA AQUI TU CÓDIGO
# data_falcon9.dropna(subset = ["LandingPad"], inplace = True) -> Para eliminar los valores none de la columna LandingPad

media = data_falcon9["PayloadMass"].mean()

nuevos_valores = data_falcon9["PayloadMass"].replace(np.nan, media)
data_falcon9["PayloadMass"] = nuevos_valores
data_falcon9

data_falcon9.isnull().sum()

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        0
Payload            0
Orbit              1
LaunchSite         0
LandingOutcome     0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad         0
Block              0
Customers          0
Mission_Outcome    1
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

El número de celdas vacías en la columna <code>PayLoadMass</code> debería cambiar a 0.


Por último tenemos que la columna `Customers` es una lista, y como hemos comentado previamente, no se llevan especialmente bien con SQL.
*Transforma esta lista en una string aplicando `.map(lambda x : ','.join(x))` a la columna y guardalo en la misma columna.*

In [ ]:
#INSERTA AQUÍ TU CÓDIGO
data_falcon9["Customers"] = data_falcon9["Customers"].map(lamba x : ','.join(x))

*Ahora exporta el dataset en formato `.csv`  con `index=False` y llámalo `dataset_part_1.csv`. Descarga el archivo porque será necesario en la siguiente práctica.*


In [216]:
#INSERTA AQUÍ TU CÓDIGO
data_falcon9.to_csv("dataset_part_1.csv", index=False)